In [200]:
import spacy
from tqdm.notebook import tqdm
import pandas as pd
import numpy as np

In [201]:
import sys
import os

src_dir = os.path.join(os.getcwd(), 'src')
sys.path.append(os.path.abspath(src_dir))

from importlib import reload

import model_2.data_type.enum

reload(model_2.data_type.enum)

from model_2.data_type.enum import CityType

In [171]:
nlp = spacy.load("fr_core_news_sm")
brute_sentences_df = pd.read_csv('../data/dataset/brute_sentences_2.csv')
cities_df = pd.read_csv('../data/cities.csv', sep=',')

In [189]:
def apply_tagging_to_sentence(sentence : str, clean_df, class_name):
    if(class_name not in sentence):
        return

    doc_sentence = nlp(sentence)
    sentence = [token_word.text for token_word in doc_sentence]
    selected_word = None

    number_of_repeats = sentence.count(class_name)
    if(number_of_repeats > 0):
        selected_word_number = np.random.randint(0, number_of_repeats)
        indexes = [idx for idx, word in enumerate(sentence) if word == class_name]
        selected_index = indexes[selected_word_number]

    

    for idx, word in enumerate(sentence):
        word_is_in_types = word in CityType._member_names_

        if(word_is_in_types):
            city = cities_df.sample()['label'].values[0].lower()

            if(word == class_name and idx == selected_index):
                city = f"[[{city}]]"
                selected_word = city
            elif(word == class_name and number_of_repeats == 0):
                city = f"[[{city}]]"
                selected_word = city

            sentence[idx] = city

    sentence = " ".join(sentence)

    clean_df.append(
        {
            "sentence": sentence,
            "class": class_name,
            "word": selected_word,
        }
    )

In [196]:
clean_df = []

with tqdm(total= 1000 * len(brute_sentences_df), unit="sentence") as pbar:
    for idx, sentence in enumerate(brute_sentences_df["sentence"]):
        for i in range(1000):

            selected_class_idx = np.random.randint(0, CityType.__len__())
            selected_class = CityType._member_names_[selected_class_idx]

            apply_tagging_to_sentence(
                sentence.lower(),
                clean_df,
                selected_class
            )

            pbar.update(1)

clean_df = pd.DataFrame(clean_df)

  0%|          | 0/152000 [00:00<?, ?sentence/s]

In [197]:
new_df_shuffled = clean_df.sample(frac=1)
train_df = new_df_shuffled[:int(len(new_df_shuffled)*0.7)]
valid_df = new_df_shuffled[int(len(new_df_shuffled)*0.7):int(len(new_df_shuffled)*0.85)]
test_df = new_df_shuffled[int(len(new_df_shuffled)*0.85):]

In [198]:
train_df.to_csv('../data/dataset/model2_train.csv', index=False)
valid_df.to_csv('../data/dataset/model2_valid.csv', index=False)
test_df.to_csv('../data/dataset/model2_test.csv', index=False)